In [1]:
import os, sys
sys.path.append('/root/catkin_ws/src/primitives/')
import pickle
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import trimesh
import networkx

from open3d import JVisualizer

import copy
import time
import argparse
import numpy as np
from multiprocessing import Process, Pipe, Queue
import pickle
import rospy
import copy
import signal
import open3d
from IPython import embed

from yacs.config import CfgNode as CN
from closed_loop_experiments import get_cfg_defaults

from airobot import Robot
from airobot.utils import pb_util
from airobot.sensor.camera.rgbdcam_pybullet import RGBDCameraPybullet
from airobot.utils import common
import pybullet as p

from helper import util
from macro_actions import ClosedLoopMacroActions, YumiGelslimPybulet
# from closed_loop_eval import SingleArmPrimitives, DualArmPrimitives

In [2]:
sys.path.append('/root/training/')

import os
import argparse
import time
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from data_loader import DataLoader
from model import VAE, GoalVAE
from util import to_var, save_state, load_net_state, load_seed, load_opt_state

import scipy.signal as signal

from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE

In [3]:
# with open ('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0/2.pkl', 'rb') as f:
#     grasp_data = pickle.load(f)
    
# print(grasp_data.keys())

In [4]:
# with open ('/root/catkin_ws/src/primitives/data/pull/face_ind_large_0_fixed/1003.pkl', 'rb') as f:
#     pull_data = pickle.load(f)
def get_rand_data(primitive='pull', train_test='test'):
    got_file = False
    while not got_file:
        val = np.random.randint(low=0, high=5000)
        if primitive == 'pull':
            test_path = '/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/%s/%d/pkl/%d.pkl' % (train_test, val, val)
        elif primitive == 'grasp':
            test_path = '/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/%s/%d/pkl/%d.pkl' % (train_test, val, val)
        if os.path.exists(test_path):
            with open (test_path, 'rb') as f:
    #             pull_data = pickle.load(f)
                data = pickle.load(f)
                got_file = True

    print(test_path)
    # print(pull_data.keys())
    print(data.keys())
    return data
pull_data = get_rand_data(primitive='pull')
grasp_data = get_rand_data(primitive='grasp')

with open('/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/metadata.pkl', 'rb') as mf:
    metadata = pickle.load(mf)
# with open('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/metadata.pkl', 'rb') as mf:
#     metadata = pickle.load(mf)


print('Metadata keys: ')
dynamics_info = metadata['dynamics']
mesh_file = metadata['mesh_file']
palm_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/mpalm/mpalms_all_coarse.stl'
table_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/table/table_top.stl'
cfg = metadata['cfg']

/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/test/1821/pkl/1821.pkl
['transformation_corrected', 'goal', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'keypoints_goal_corrected']
/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/test/232/pkl/232.pkl
['transformation_corrected', 'goal', 'keypoints_goal_corrected', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'goal_face']
Metadata keys: 


# Viz Utils

In [280]:
def vis_palms(data, name='pull', goal='transformation', goal_number=1, palm_number=1):
    obj_mesh = trimesh.load_mesh(mesh_file)
    table_mesh = trimesh.load_mesh(table_mesh_file)
    
    obj_pos_world = data['start'][:3]
    obj_ori_world = data['start'][3:]
    obj_ori_mat = common.quat2rot(obj_ori_world)
    h_trans = np.zeros((4, 4))
    h_trans[:3, :3] = obj_ori_mat
    h_trans[:-1, -1] = obj_pos_world
    h_trans[-1, -1] = 1

    obj_mesh.apply_transform(h_trans)
    
    if len(data['transformation_corrected']) > 0:
        if not isinstance(data['transformation_corrected'][0], list):
            tmp = data['transformation_corrected']
            data['transformation_corrected'] = [tmp]
    else:
        goal_number = 0
    if not isinstance(data['contact_obj_frame']['right'][0], list):
        tmp_r = data['contact_obj_frame']['right']
        tmp_l = data['contact_obj_frame']['left']
        data['contact_obj_frame']['right'] = [tmp_r]
        data['contact_obj_frame']['left'] = [tmp_l]
        
    
    goal_obj_meshes = []
    for i in range(goal_number):
        goal_obj_mesh = trimesh.load_mesh(mesh_file)        
        if goal == 'transformation':
            T_mat = util.matrix_from_pose(util.list2pose_stamped(data['transformation_corrected'][i]))
            T_start = util.matrix_from_pose(util.list2pose_stamped(data['start']))
            T_goal = np.matmul(T_mat, T_start)
            goal_obj_pose_world = util.pose_stamped2list(util.pose_from_matrix(T_goal))
    #         goal_obj_pose_start = util.list2pose_stamped(data['transformation_corrected'])
    #         goal_obj_pose_world = util.pose_stamped2list(
    #             util.convert_reference_frame(
    #                 goal_obj_pose_start,
    #                 util.unit_pose(),
    #                 util.list2pose_stamped(data['start'])))
            goal_obj_pos_world = goal_obj_pose_world[:3]
            goal_obj_ori_world = goal_obj_pose_world[3:]
        else:
            goal_obj_pos_world = data['goal'][:3]
            goal_obj_ori_world = data['goal'][3:]
        goal_obj_ori_mat = common.quat2rot(goal_obj_ori_world)
        goal_h_trans = np.zeros((4, 4))
        goal_h_trans[:3, :3] = goal_obj_ori_mat
        goal_h_trans[:-1, -1] = goal_obj_pos_world
        goal_h_trans[-1, -1] = 1 

        goal_obj_mesh.apply_transform(goal_h_trans)
        goal_obj_meshes.append(goal_obj_mesh)
    
    r_palm_meshes = []
    l_palm_meshes = []
    
    if name == 'pull':
        for j in range(palm_number):
            r_palm_mesh = trimesh.load_mesh(palm_mesh_file)
            l_palm_mesh = trimesh.load_mesh(palm_mesh_file)
            
            tip_contact_r_obj = util.list2pose_stamped(data['contact_obj_frame']['right'][j])
            tip_contact_r = util.convert_reference_frame(
                pose_source=tip_contact_r_obj,
                pose_frame_target=util.unit_pose(),
                pose_frame_source=util.list2pose_stamped(data['start']))

            wrist_contact_r = util.convert_reference_frame(
                pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
                pose_frame_target=util.unit_pose(),
                pose_frame_source=tip_contact_r)

            wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)

            palm_pos_world_r = wrist_contact_r_list[:3]
            palm_ori_world_r = wrist_contact_r_list[3:]
            palm_ori_mat = common.quat2rot(palm_ori_world_r)
            h_trans = np.zeros((4, 4))
            h_trans[:3, :3] = palm_ori_mat
            h_trans[:-1, -1] = palm_pos_world_r
            h_trans[-1, -1] = 1

            r_palm_mesh.apply_transform(h_trans)
            r_palm_meshes.append(r_palm_mesh)
        
#         scene = trimesh.Scene([obj_mesh, r_palm_mesh, table_mesh, goal_obj_mesh]) 
#         scene = trimesh.Scene([obj_mesh, r_palm_mesh, table_mesh] + goal_obj_meshes)
        scene = trimesh.Scene([obj_mesh, table_mesh] + r_palm_meshes + goal_obj_meshes)
    else:
        for j in range(palm_number):
            r_palm_mesh = trimesh.load_mesh(palm_mesh_file)
            l_palm_mesh = trimesh.load_mesh(palm_mesh_file)
            
            tip_contact_r_obj = util.list2pose_stamped(data['contact_obj_frame']['right'][j])
            tip_contact_l_obj = util.list2pose_stamped(data['contact_obj_frame']['left'][j])

            tip_contact_r = util.convert_reference_frame(
                pose_source=tip_contact_r_obj,
                pose_frame_target=util.unit_pose(),
                pose_frame_source=util.list2pose_stamped(data['start']))

            tip_contact_l = util.convert_reference_frame(
                pose_source=tip_contact_l_obj,
                pose_frame_target=util.unit_pose(),
                pose_frame_source=util.list2pose_stamped(data['start']))            

            wrist_contact_r = util.convert_reference_frame(
                pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
                pose_frame_target=util.unit_pose(),
                pose_frame_source=tip_contact_r)

            wrist_contact_l = util.convert_reference_frame(
                pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
                pose_frame_target=util.unit_pose(),
                pose_frame_source=tip_contact_l)

            wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)
            wrist_contact_l_list = util.pose_stamped2list(wrist_contact_l)

            palm_pos_world_r = wrist_contact_r_list[:3]
            palm_ori_world_r = wrist_contact_r_list[3:]
            palm_ori_mat = common.quat2rot(palm_ori_world_r)
            h_trans = np.zeros((4, 4))
            h_trans[:3, :3] = palm_ori_mat
            h_trans[:-1, -1] = palm_pos_world_r
            h_trans[-1, -1] = 1

            r_palm_mesh.apply_transform(h_trans)

            palm_pos_world_l = wrist_contact_l_list[:3]
            palm_ori_world_l = wrist_contact_l_list[3:]
            palm_ori_mat = common.quat2rot(palm_ori_world_l)
            h_trans = np.zeros((4, 4))
            h_trans[:3, :3] = palm_ori_mat
            h_trans[:-1, -1] = palm_pos_world_l
            h_trans[-1, -1] = 1

            l_palm_mesh.apply_transform(h_trans)        

            r_palm_meshes.append(r_palm_mesh)
            l_palm_meshes.append(l_palm_mesh)

#         scene = trimesh.Scene([obj_mesh, r_palm_mesh, l_palm_mesh, table_mesh, goal_obj_mesh])
#         scene = trimesh.Scene([obj_mesh, r_palm_mesh, l_palm_mesh, table_mesh] + goal_obj_meshes)
        scene = trimesh.Scene([obj_mesh, table_mesh] + r_palm_meshes + l_palm_meshes + goal_obj_meshes)
#         scene = trimesh.Scene([obj_mesh, r_palm_mesh, table_mesh])
    good_camera_euler = [1.0513555,  -0.02236318, -1.62958927]
    box_count = 0
    for key in scene.geometry.keys():
        print(key)
        if 'mpalms_all_coarse' in key:
            scene.geometry[key].visual.face_colors = [100, 100, 0, 30]
        if 'realsense_box_experiments' in key:
            if box_count == 0:
                scene.geometry[key].visual.face_colors = [250, 200, 200, 150]
                box_count += 1
            else:
                scene.geometry[key].visual.face_colors = [200, 200, 250, 150]                

    scene.geometry['table_top.stl'].visual.face_colors = [200, 200, 200, 250]
#     scene.geometry['realsense_box_experiments.stl'].visual.face_colors = [200, 200, 250, 250]
    scene.set_camera(angles=good_camera_euler, center=data['start'][:3], distance=0.8)
    return scene

def correct_pos(data):
    contact_obj_frame_pred = util.list2pose_stamped(pull_data_eval['contact_obj_frame'])
    contact_world_frame = util.convert_reference_frame(contact_obj_frame_pred, util.unit_pose(), util.list2pose_stamped(start))
    contact_pos = open3d.utility.DoubleVector(np.array(util.pose_stamped2list(contact_world_frame)[:3]))

    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_pts']))
    pcd.colors = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_colors']) / 255.0)

    kdtree = open3d.geometry.KDTreeFlann(pcd)
    nearest_pt_ind = kdtree.search_knn_vector_3d(contact_pos, 1)[1][0]
    nearest_pt_world = np.asarray(pcd.points)[nearest_pt_ind]

    contact_world_frame_corrected = copy.deepcopy(contact_world_frame)
    contact_world_frame_corrected.pose.position.x = nearest_pt_world[0]
    contact_world_frame_corrected.pose.position.y = nearest_pt_world[1]
    contact_world_frame_corrected.pose.position.z = nearest_pt_world[2]

    contact_obj_frame_corrected = util.pose_stamped2list(util.convert_reference_frame(contact_world_frame_corrected, util.list2pose_stamped(start), util.unit_pose()))
    new_data = copy.deepcopy(data)
    new_data['contact_obj_frame'] = contact_obj_frame_corrected
    return new_data


def plot_keypoint_goal(data):

    i_list=[0, 1, 4, 5, 2, 3, 0, 5, 1, 1, 0, 2]
    j_list=[1, 2, 5, 6, 3, 6, 1, 1, 5, 3, 4, 4]
    k_list=[2, 3, 6, 7, 6, 7, 4, 4, 7, 7, 6, 6]
    
    red_marker = {
        'size' : 1.0,
        'color' : 'red',                # set color to an array/list of desired values
        'colorscale' : 'Viridis',   # choose a colorscale
        'opacity' : 0.8
    } 
    blue_marker = {
        'size' : 1.0,
        'color' : 'blue',                # set color to an array/list of desired values
        'colorscale' : 'Viridis',   # choose a colorscale
        'opacity' : 0.8
    }   
    
    black_marker = {
        'size' : 3.0,
        'color' : 'black',                # set color to an array/list of desired values
        'colorscale' : 'Viridis',   # choose a colorscale
        'opacity' : 0.8
    }       
    
    kp_start = data['keypoints_start']

    start_data = {
        'type': 'mesh3d',
        'x': kp_start[:, 0],
        'y': kp_start[:, 1],
        'z': kp_start[:, 2],
        'color': 'red',
        'opacity': 0.95,
        'delaunayaxis': 'z',
        'i': i_list,
        'j': j_list,
        'k': k_list,
        'flatshading': True
    }
    
    start_kp_data = {
        'type': 'scatter3d',
        'x': kp_start[:, 0],
        'y': kp_start[:, 1],
        'z': kp_start[:, 2],
        'mode': 'markers',
        'marker': red_marker        
    }
    
    plane_data = {
        'type': 'mesh3d',
        'x': [-1, 1, 1, -1],
        'y': [-1, -1, 1, 1],
        'z': [0, 0, 0, 0],
        'color': 'gray',
        'opacity': 0.5,
        'delaunayaxis': 'z'
    }
    
    fig_data = []   
    fig_data.append(start_data)    
    fig_data.append(plane_data)
    
    
    if isinstance(data['keypoints_goal_corrected'], list):        
        for kp_goal in data['keypoints_goal_corrected']:
            goal_data = {
                'type': 'mesh3d',
                'x': kp_goal[:, 0],
                'y': kp_goal[:, 1],
                'z': kp_goal[:, 2],
                'color': 'blue',
                'opacity': 0.05,
                'delaunayaxis': 'z',
                'i': i_list,
                'j': j_list,
                'k': k_list,
                'flatshading': True
            }
            
            goal_kp_data = {
                'type': 'scatter3d',
                'x': kp_goal[:, 0],
                'y': kp_goal[:, 1],
                'z': kp_goal[:, 2],
                'mode': 'markers',
                'marker': black_marker
            }
            
            fig_data.append(goal_data)
            fig_data.append(goal_kp_data)
    else:
        kp_goal = data['keypoints_goal_corrected']
        goal_data = {
            'type': 'mesh3d',
            'x': kp_goal[:, 0],
            'y': kp_goal[:, 1],
            'z': kp_goal[:, 2],
            'color': 'blue',
            'opacity': 0.05,
            'delaunayaxis': 'z',
            'i': i_list,
            'j': j_list,
            'k': k_list,
            'flatshading': True
        }
        
        goal_kp_data = {
            'type': 'scatter3d',
            'x': kp_goal[:, 0],
            'y': kp_goal[:, 1],
            'z': kp_goal[:, 2],
            'mode': 'markers',
            'marker': black_marker
        }       
        
        fig_data.append(goal_data)
        fig_data.append(goal_kp_data)

    fig = go.Figure(data=fig_data)
    
#     camera = {
#         'up': {'x': 0, 'y': 0,'z': 1},
#         'center': {'x': data['start'][0], 'y': data['start'][1], 'z': data['start'][2]-0.5},
#         'eye': {'x': -1.0, 'y': 0.0, 'z': 0.01}
#     } 
    camera = {
        'up': {'x': 0, 'y': 0,'z': 1},
        'center': {'x': 0.45, 'y': 0, 'z': 0.0},
        'eye': {'x': -1.0, 'y': 0.0, 'z': 0.01}
    } 

    scene = {
        'xaxis': {'nticks': 10, 'range': [-0.1, 0.9]},
        'yaxis': {'nticks': 16, 'range': [-0.5, 0.5]},
        'zaxis': {'nticks': 8, 'range': [-0.01, 0.99]}
    }
    width = 700
    margin = {'r': 20, 'l': 10, 'b': 10, 't': 10}    
    
    fig.update_layout(
        scene=scene, 
        scene_camera=camera,
        width=width,
        margin=margin
    )    
    return fig

def plot_keypoint_plan(keypoint_plan):

    i_list=[0, 1, 4, 5, 2, 3, 0, 5, 1, 1, 0, 2]
    j_list=[1, 2, 5, 6, 3, 6, 1, 1, 5, 3, 4, 4]
    k_list=[2, 3, 6, 7, 6, 7, 4, 4, 7, 7, 6, 6]
    
    red_marker = {
        'size' : 1.0,
        'color' : 'red',                # set color to an array/list of desired values
        'colorscale' : 'Viridis',   # choose a colorscale
        'opacity' : 0.8
    } 
    blue_marker = {
        'size' : 1.0,
        'color' : 'blue',                # set color to an array/list of desired values
        'colorscale' : 'Viridis',   # choose a colorscale
        'opacity' : 0.8
    }   
    
    black_marker = {
        'size' : 3.0,
        'color' : 'black',                # set color to an array/list of desired values
        'colorscale' : 'Viridis',   # choose a colorscale
        'opacity' : 0.8
    }       
    
    kp_start = keypoint_plan[0]['keypoints_start']

    start_data = {
        'type': 'mesh3d',
        'x': kp_start[:, 0],
        'y': kp_start[:, 1],
        'z': kp_start[:, 2],
        'color': 'red',
        'opacity': 0.95,
        'delaunayaxis': 'z',
        'i': i_list,
        'j': j_list,
        'k': k_list,
        'flatshading': True
    }
    
    start_kp_data = {
        'type': 'scatter3d',
        'x': kp_start[:, 0],
        'y': kp_start[:, 1],
        'z': kp_start[:, 2],
        'mode': 'markers',
        'marker': red_marker        
    }
    
    plane_data = {
        'type': 'mesh3d',
        'x': [-1, 1, 1, -1],
        'y': [-1, -1, 1, 1],
        'z': [0, 0, 0, 0],
        'color': 'gray',
        'opacity': 0.5,
        'delaunayaxis': 'z'
    }
    
    fig_data = []   
    fig_data.append(start_data)    
    fig_data.append(plane_data)
    
    for i in range(len(keypoint_plan) - 1):
        kp_goal = keypoint_plan[i+1]['keypoints_start']
        
        goal_data = {
            'type': 'mesh3d',
            'x': kp_goal[:, 0],
            'y': kp_goal[:, 1],
            'z': kp_goal[:, 2],
            'color': 'blue',
            'opacity': 0.05,
            'delaunayaxis': 'z',
            'i': i_list,
            'j': j_list,
            'k': k_list,
            'flatshading': True
        }
        goal_kp_data = {
            'type': 'scatter3d',
            'x': kp_goal[:, 0],
            'y': kp_goal[:, 1],
            'z': kp_goal[:, 2],
            'mode': 'markers',
            'marker': black_marker
        }
        fig_data.append(goal_data)
        fig_data.append(goal_kp_data)
    
    fig = go.Figure(data=fig_data)
    
    camera = {
        'up': {'x': 0, 'y': 0,'z': 1},
        'center': {'x': 0.45, 'y': 0, 'z': 0.0},
        'eye': {'x': -1.0, 'y': 0.0, 'z': 0.01}
    }    

    scene = {
        'xaxis': {'nticks': 10, 'range': [-0.1, 0.9]},
        'yaxis': {'nticks': 16, 'range': [-0.5, 0.5]},
        'zaxis': {'nticks': 8, 'range': [-0.01, 0.99]}
    }
    width = 700
    margin = {'r': 20, 'l': 10, 'b': 10, 't': 10}    
    
    fig.update_layout(
        scene=scene, 
        scene_camera=camera,
        width=width,
        margin=margin
    )    
    return fig

def transform_keypoints(data, number=1):
    if not isinstance(data['transformation_corrected'][0], list):
        tmp = data['transformation_corrected']
        data['transformation_corrected'] = [tmp]
    
    data['keypoints_goal_corrected'] = []
    
    keypoints_start_homog = np.hstack(
        (data['keypoints_start'], np.ones(
            (data['keypoints_start'].shape[0], 1)))
    )    
    
    for i in range(number):
        T_mat = util.matrix_from_pose(util.list2pose_stamped(data['transformation_corrected'][i]))
        data['keypoints_goal_corrected'].append(np.matmul(T_mat, keypoints_start_homog.T).T[:, :3])
    return data

def transform_keypoints_full(data, number=1):
    if not isinstance(data['transformation_corrected'][0], list):
        tmp = data['transformation_corrected']
        data['transformation_corrected'] = [tmp]
    
    data['keypoints_goal_corrected'] = []
    data['goal'] = []
    
    keypoints_start_homog = np.hstack(
        (data['keypoints_start'], np.ones(
            (data['keypoints_start'].shape[0], 1)))
    )
    
    T_start = util.matrix_from_pose(util.list2pose_stamped(data['start']))
    
    for i in range(number):
        T_mat = util.matrix_from_pose(util.list2pose_stamped(data['transformation_corrected'][i]))
        data['keypoints_goal_corrected'].append(np.matmul(T_mat, keypoints_start_homog.T).T[:, :3])
        T_goal = np.matmul(T_mat, T_start)        
        data['goal'].append(util.pose_stamped2list(util.pose_from_matrix(T_goal)))
    return data


In [49]:
# scene = vis_palms(grasp_data, name='grasp', goal='goal')
scene = vis_palms(get_rand_data('grasp'), name='grasp', goal='transformation')

scene.show(viewer='gl')

/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/test/2090/pkl/2090.pkl
['transformation_corrected', 'goal', 'keypoints_goal_corrected', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'goal_face']
realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
mpalms_all_coarse.stl:EUTNDPMOQYFJ
realsense_box_experiments.stl:EUVG1YM6BRJC


SceneViewer(width=1800, height=1350)

# Torch Utils

In [269]:
def torch_pose_to_list(palm_pose_tensor):
    pos = palm_pose_tensor[:3].data.cpu().numpy()
    ori = palm_pose_tensor[3:].data.cpu().numpy()
    
    ori = ori/np.linalg.norm(ori)
    
    return pos.tolist() + ori.tolist()

def to_torch_contact(data):
    inp0 = torch.from_numpy(data['keypoints_start'].flatten().astype(np.float32)).cuda()
    inp1 = torch.from_numpy(data['keypoints_goal_corrected'].flatten().astype(np.float32)).cuda()
    return torch.cat((inp0, inp1))

def to_torch_trans(data):
    inp0 = torch.from_numpy(data['keypoints_start'].flatten().astype(np.float32)).cuda()
    return inp0

def to_torch_trans_full(data):
    inp0 = torch.from_numpy(data['keypoints_start'].flatten().astype(np.float32)).cuda()
    inp1 = torch.from_numpy(np.asarray(data['transformation_corrected'], dtype=np.float32)).cuda()
    return inp0, inp1

def eval_grasp_trans_sample(keypoints_start, trans_sample, grasp_vae, grasp_clf):
    inp = torch.cat((keypoints_start, trans_sample))
    z_sample = grasp_vae.encode(inp)[1]
    
    log_prob = grasp_clf.score(z_sample.data.cpu().numpy().reshape(1, -1))
    return log_prob

def eval_contact_sample(keypoints_start, keypoints_goal, palm_sample, vae, clf):
    inp = torch.cat((keypoints_start, keypoints_goal, palm_sample))
    z_sample = vae.encode(inp)[1]
    
    log_prob = clf.score(z_sample.data.cpu().numpy().reshape(1, -1))
    return log_prob

def empty_state():
    data = {}
    data['keypoints_start'] = None
    data['keypoints_goal_corrected'] = []
    data['transformation_corrected'] = []
    data['start'] = None
    data['goal'] = None
    data['contact_obj_frame'] = {}
    data['contact_obj_frame']['right'] = None
    data['contact_obj_frame']['left'] = None
    return data

In [77]:
def sample_contact(vae, start_data, primitive='pull', num_samples=1, clf=None):
    output_data = copy.deepcopy(start_data)
    output_data['contact_obj_frame']['right'] = []
    output_data['contact_obj_frame']['left'] = []    
    
    x = to_torch_contact(start_data)
    for i in range(num_samples):
        if clf is None:
            z_sample = torch.randn_like(torch.zeros(3,)).cuda()
        else:
            z_sample = torch.from_numpy(clf.sample()[0].squeeze()).cuda()

        palm_poses = vae.decode(z_sample, x)
        r_palm, l_palm = palm_poses[0], palm_poses[1]

        output_data['contact_obj_frame']['right'].append(torch_pose_to_list(r_palm))
        output_data['contact_obj_frame']['left'].append(torch_pose_to_list(l_palm))
        
    return output_data

def sample_pull_transformation(pull_vae, start_data, num_samples=1, pull_clf=None):
    x = torch.from_numpy(start_data['keypoints_start'].flatten().astype(np.float32)).cuda()
    
    output_data = copy.deepcopy(start_data)
    output_data['transformation_corrected'] = []
    output_data['keypoints_goal_corrected'] = []    
    for i in range(num_samples):
        z_sample = torch.randn_like(torch.zeros(3,)).cuda()
        trans = pull_vae.decode(z_sample, x)
        
        output_data['transformation_corrected'].append(pose_to_list(trans))
    
    output_data = transform_keypoints(output_data, number=num_samples)
    
    return output_data

def sample_grasp_transformation(grasp_vae, grasp_clf, start_data, num_samples=1, max_k=100, best_threshold=9.0):
    x = torch.from_numpy(start_data['keypoints_start'].flatten().astype(np.float32)).cuda()
    
    output_data = copy.deepcopy(start_data)
    output_data['transformation_corrected'] = []
    output_data['keypoints_goal_corrected'] = []
    for i in range(num_samples):
        k = 0
        done = False
        best_so_far = 0.0
        best_trans = None
        while not done:
            k += 1
            
            z_sample = torch.randn_like(torch.zeros(3,)).cuda()
#             z_sample = torch.from_numpy(grasp_clf.sample()[0].squeeze().astype(np.float32)).cuda()
            trans = grasp_vae.decode(z_sample, x)
            if best_trans is None:
                best_trans = trans
    
            log_prob = eval_grasp_trans_sample(x, trans, grasp_vae, grasp_clf)
            
            if log_prob > best_so_far:
                best_so_far = log_prob
                best_trans = trans
        
            if best_so_far > best_threshold or k >= max_k:
                break
                
        output_data['transformation_corrected'].append(pose_to_list(trans))
    output_data = transform_keypoints(output_data, number=num_samples)
    return output_data

# Load data

In [ ]:
grasp_loader = DataLoader(data_dir = '/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/train/')
grasp_loader.create_random_ordering()

pull_loader = DataLoader(data_dir = '/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/train/')
pull_loader.create_random_ordering()

In [107]:
pull_data = get_rand_data(primitive='pull', train_test='test')
vis_palms(pull_data, name='pull').show(viewer='gl')
pull_data_eval = copy.deepcopy(pull_data)

/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/test/1802/pkl/1802.pkl
['transformation_corrected', 'goal', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'keypoints_goal_corrected']
realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
realsense_box_experiments.stl:BZVQR5V0OKVJ


In [110]:
grasp_data = get_rand_data(primitive='grasp', train_test='test')
vis_palms(grasp_data, name='grasp').show(viewer='gl')
grasp_data_eval = copy.deepcopy(grasp_data)

/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/test/4098/pkl/4098.pkl
['transformation_corrected', 'goal', 'keypoints_goal_corrected', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'goal_face']
realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
mpalms_all_coarse.stl:N7Y6RQHWDQTV
realsense_box_experiments.stl:N7BWCOHAP1NZ


# Load models and fit latent space PDF

In [204]:
grasp_palm_vae = VAE(48+14, 7, 3, 48, [512, 256, 128, 64], 0.0003)
load_net_state(grasp_palm_vae, '/root/training/saved_models/grasp_contact_keypoints_start_goal_fixed_full_data_anneal_batch128_fixed/grasp_contact_keypoints_start_goal_fixed_full_data_anneal_batch128_fixed_epoch_100.pt')

grasp_palm_vae.encoder.cuda()
grasp_palm_vae.decoder.cuda()


dataset = grasp_loader.load_dataset(start_rep='keypoints', goal_rep='keypoints', task='contact')
x = torch.from_numpy(dataset[0]).cuda()
z, z_mu, z_logvar = grasp_palm_vae.encode(x)

x_train = z_mu.data.cpu().numpy()

grasp_palm_clf = GaussianMixture(n_components=10, covariance_type='full')
grasp_palm_clf.fit(x_train)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [64]:
# grasp_palm_vae = VAE(48+14, 7, 3, 48, [512, 256, 128, 64], 0.0003)
# load_net_state(grasp_palm_vae, '/root/training/saved_models/grasp_contact_keypoints_start_goal_fixed_full_data_anneal_batch128_fixed/grasp_contact_keypoints_start_goal_fixed_full_data_anneal_batch128_fixed_epoch_150.pt')

# grasp_palm_vae.encoder.cuda()
# grasp_palm_vae.decoder.cuda()

Decoder(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=512, bias=True)
  )
  (in_fc): Linear(in_features=51, out_features=64, bias=True)
  (out_right_head): Linear(in_features=512, out_features=7, bias=True)
  (out_left_head): Linear(in_features=512, out_features=7, bias=True)
)

In [224]:
pull_trans_vae = GoalVAE(31, 7, 3, 24, [512, 256, 128, 64], 0.0003)
# load_net_state(pull_trans_vae, '/root/training/saved_models/pull_transformation_full_data_start_keypoints_cond_batch_512_fixed_1/pull_transformation_full_data_start_keypoints_cond_batch_512_fixed_1_epoch_50.pt')
# load_net_state(pull_trans_vae, '/root/training/saved_models/pull_transformation_full_data_start_keypoints_cond_batch_512_fixed_xy_relax_0/pull_transformation_full_data_start_keypoints_cond_batch_512_fixed_xy_relax_0_epoch_150.pt')
load_net_state(pull_trans_vae, '/root/training/saved_models/pull_transformation_full_data_start_keypoints_cond_batch_512_fixed_no_xy_0/pull_transformation_full_data_start_keypoints_cond_batch_512_fixed_no_xy_0_epoch_150.pt')

pull_trans_vae.encoder.cuda()
pull_trans_vae.decoder.cuda()


dataset = pull_loader.load_dataset(start_rep='keypoints', goal_rep='keypoints', task='transformation')
x = torch.from_numpy(dataset[0]).cuda()
z, z_mu, z_logvar = pull_trans_vae.encode(x)

x_train = z_mu.data.cpu().numpy()

pull_trans_clf = GaussianMixture(n_components=10, covariance_type='full')
pull_trans_clf.fit(x_train)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [101]:
grasp_trans_vae = GoalVAE(31, 7, 3, 24, [512, 256, 128, 64], 0.0003)
load_net_state(grasp_trans_vae, '/root/training/saved_models/grasp_transformation_cond_start_full_data_fixed_anneal_0/grasp_transformation_cond_start_full_data_fixed_anneal_0_epoch_980.pt')

grasp_trans_vae.encoder.cuda()
grasp_trans_vae.decoder.cuda()

dataset = grasp_loader.load_dataset(start_rep='keypoints', goal_rep='keypoints', task='transformation')
x = torch.from_numpy(dataset[0]).cuda()
z, z_mu, z_logvar = grasp_trans_vae.encode(x)

x_train = z_mu.data.cpu().numpy()

grasp_trans_clf = GaussianMixture(n_components=10, covariance_type='full')
grasp_trans_clf.fit(x_train)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [67]:
# pull_contact_out = sample_contact(pull_palm_vae, pull_data_eval, primitive='pull', num_samples=10)

# vis_palms(pull_contact_out, name='pull', goal='transformation', palm_number=10).show(viewer='gl')

In [104]:
grasp_contact_out = sample_contact(grasp_palm_vae, grasp_data_eval, primitive='grasp', num_samples=10)

vis_palms(grasp_contact_out, name='grasp', goal='transformation', palm_number=10).show(viewer='gl')

realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
mpalms_all_coarse.stl:NZIUMYMMZTUP
mpalms_all_coarse.stl:NZJ4NKTVZO3P
mpalms_all_coarse.stl:NZGYRSM9JABF
mpalms_all_coarse.stl:NZHA45SXNHEZ
mpalms_all_coarse.stl:NZGEIGQKWW3Y
mpalms_all_coarse.stl:NZIFY3BPNAZG
mpalms_all_coarse.stl:NZIYDGSEEQ9P
mpalms_all_coarse.stl:NZI9CE2NQXPP
mpalms_all_coarse.stl:NZH4NMNIH2BN
mpalms_all_coarse.stl:NZIKLSLAUQPP
mpalms_all_coarse.stl:NZHWHT9OHWFU
mpalms_all_coarse.stl:NZHJ7QSGIBXU
mpalms_all_coarse.stl:NZHOLO7THOTP
mpalms_all_coarse.stl:NZJWCIJZKBFP
mpalms_all_coarse.stl:NZHUZH691ERT
mpalms_all_coarse.stl:NZJJUTDMSBFJ
mpalms_all_coarse.stl:NZHSTECZD3TP
mpalms_all_coarse.stl:NZH1GZKRHOPW
mpalms_all_coarse.stl:NZIIPDRO5YBN
realsense_box_experiments.stl:NZGCO82DPYDE


SceneViewer(width=1800, height=1350)

In [225]:
out_pull = sample_pull_transformation(pull_trans_vae, pull_data_0, num_samples=20)

pull_fig = plot_keypoint_goal(out_pull)
pull_fig.show()

In [109]:
vis_palms(out_pull, name='pull', goal='transformation',goal_number=10).show(viewer='gl')

realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
realsense_box_experiments.stl:AZTCCC7DFWJX
realsense_box_experiments.stl:AZU6PCVIHW7V
realsense_box_experiments.stl:AZVPTV9HN6XN
realsense_box_experiments.stl:AZUFLJPEQ4DM
realsense_box_experiments.stl:AZVW17MTSRZR
realsense_box_experiments.stl:AZVXZGGTOTQP
realsense_box_experiments.stl:AZUPF7RRFHDS
realsense_box_experiments.stl:AZVSUEQGJXTP
realsense_box_experiments.stl:AZVC8IJNJHHK
realsense_box_experiments.stl:AZUGULQW5IBY


SceneViewer(width=1800, height=1350)

In [111]:
out_grasp = sample_grasp_transformation(grasp_trans_vae, grasp_trans_clf, grasp_data_eval, num_samples=10)

grasp_fig = plot_keypoint_goal(out_grasp)
grasp_fig.show()

# Planning sequences

In [262]:
pull_data_0 = get_rand_data(primitive='pull', train_test='test')
vis_palms(pull_data_0, name='pull').show(viewer='gl')
pull_data_eval_0 = copy.deepcopy(pull_data_0)

/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/test/196/pkl/196.pkl
['transformation_corrected', 'goal', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'keypoints_goal_corrected']
realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
realsense_box_experiments.stl:4QPMYQCEBGTF


In [182]:
pull_data_1 = get_rand_data(primitive='pull', train_test='test')
vis_palms(pull_data_1, name='pull').show(viewer='gl')
pull_data_eval_1 = copy.deepcopy(pull_data_1)

/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/test/4893/pkl/4893.pkl
['transformation_corrected', 'goal', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'keypoints_goal_corrected']
realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
realsense_box_experiments.stl:4FA4F8ZVM0XY


In [183]:
grasp_data = get_rand_data(primitive='grasp', train_test='test')
vis_palms(grasp_data, name='grasp').show(viewer='gl')
grasp_data_eval = copy.deepcopy(grasp_data)

/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/test/4569/pkl/4569.pkl
['transformation_corrected', 'goal', 'keypoints_goal_corrected', 'contact_obj_frame', 'contact_world_frame', 'start', 'result', 'keypoints_start', 'contact_pcd', 'keypoints_goal', 'transformation', 'goal_face']
realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
mpalms_all_coarse.stl:FYEAVSA5V7VP
realsense_box_experiments.stl:FY2BTM83QJMP


In [184]:
def at_goal(current, goal, tol=0.003):
    center_current_xyz = np.mean(current['keypoints_start'], axis=0)
    center_goal_xyz = np.mean(goal['keypoints_goal_corrected'], axis=0)
    
    dist = np.linalg.norm(center_current_xyz - center_goal_xyz)
    if dist < tol:
        return True, dist
    else:
        return False, dist

In [283]:
start_state = copy.deepcopy(pull_data_0)
goal_state = copy.deepcopy(pull_data_1)
grasp_goal = {}
grasp_goal['keypoints_goal_corrected'] = grasp_data_eval['keypoints_start']
keypoint_plan = []
keypoint_plan.append(start_state)

done = False
k = 0
max_k = 30
best_dist = 1.0
best_score = -100.0

nodes = 20
executed_grasp = False

current_state = start_state
while not done:
    
    # get new pull transformations from current
    new_pull_state = sample_pull_transformation(
        pull_vae=pull_trans_vae, 
        start_data=current_state,
        num_samples=nodes)
#     new_pull_state = transform_keypoints(new_pull_state, number=nodes)
    new_pull_state = transform_keypoints_full(new_pull_state, number=nodes)
    
    # get new grasp transformations from current
    new_grasp_state = sample_grasp_transformation(
        grasp_vae=grasp_trans_vae, 
        grasp_clf=grasp_trans_clf,
        start_data=current_state,
        num_samples=nodes)
#     new_grasp_state = transform_keypoints(new_grasp_state, number=nodes)
    new_grasp_state = transform_keypoints_full(new_grasp_state, number=nodes)
    new_grasp_state_scores = []
    
    # evaluate grasp sample from current
#     keypoints_start, trans = to_torch_trans_full(new_grasp_state)
#     for i in range(nodes):
#         log_prob = eval_grasp_trans_sample(
#             keypoints_start=keypoints_start, 
#             trans_sample=trans[i, :], 
#             grasp_vae=grasp_trans_vae, 
#             grasp_clf=grasp_trans_clf)
#         new_grasp_state_scores.append(log_prob)
        
#     print("Scores from current: ")
#     for i, score in enumerate(new_grasp_state_scores):
#         print(i, score)

    # sample grasps for each node expanded from pulling
    grasp_states_post_pull = []
    grasp_state_post_pull_dists = []
    grasp_state_post_pull_log_probs = []
    for i in range(nodes):
        pull_state_expand = {}
        pull_state_expand['keypoints_start'] = new_pull_state['keypoints_goal_corrected'][i]
        pull_state_expand['transformation_corrected'] = [new_pull_state['transformation_corrected'][i]]
        _, dist = at_goal(pull_state_expand, grasp_goal)

        if dist < 0.05:
            grasp_state_post_pull = sample_grasp_transformation(
                grasp_vae=grasp_trans_vae,
                grasp_clf=grasp_trans_clf,
                start_data=pull_state_expand,
                num_samples=1
            )
#             grasp_state_post_pull = transform_keypoints(grasp_state_post_pull)
            grasp_state_post_pull = transform_keypoints_full(grasp_state_post_pull)

            keypoints_start, trans = to_torch_trans_full(grasp_state_post_pull)

            log_prob = eval_grasp_trans_sample(
                keypoints_start=keypoints_start,
                trans_sample=trans[0],
                grasp_vae=grasp_trans_vae,
                grasp_clf=grasp_trans_clf)
            grasp_state_post_pull_log_probs.append(log_prob)
        else:
            grasp_state_post_pull_log_probs.append(-100.0)
        
        grasp_states_post_pull.append(grasp_state_post_pull)
        grasp_state_post_pull_dists.append(dist)
        
    if len(grasp_state_post_pull_log_probs) > 0:
        print("log probs after pulling: ")
        closest_score_ind = np.argmax(grasp_state_post_pull_log_probs)
        closest_score = grasp_state_post_pull_log_probs[closest_score_ind]        
        for i, log_prob in enumerate(grasp_state_post_pull_log_probs):
            if log_prob > 0.0:
                print(i, log_prob)
    else:
        closest_score = -1.0

#     print("dists after pulling: ")
#     for i, dist in enumerate(grasp_state_post_pull_dists):
#         print(i, dist)
        
    closest_dist_ind = np.argmin(grasp_state_post_pull_dists)    
    closest_dist = grasp_state_post_pull_dists[closest_dist_ind]

    prev_state = copy.deepcopy(current_state)
    current_state = {}
#     if closest_score > 11.0:
#     if closest_dist < 0.03 and closest_score > 15.0:
    if closest_score > 9.0:
        print("executed grasp!")
#         current_state['keypoints_start'] = grasp_states_post_pull[closest_dist_ind]['keypoints_goal_corrected'][0]
        current_state['keypoints_start'] = grasp_states_post_pull[closest_score_ind]['keypoints_goal_corrected'][0]
        current_state['start'] = grasp_states_post_pull[closest_score_ind]['goal'][0]
        executed_grasp = True
    else:
#         if closest_dist < best_dist:
        if closest_score > best_score:
#             best_dist = closest_dist
            best_score = closest_score
#             current_state['keypoints_start'] = new_pull_state['keypoints_goal_corrected'][closest_dist_ind]
            current_state['keypoints_start'] = new_pull_state['keypoints_goal_corrected'][closest_score_ind]
            current_state['start'] = new_pull_state['goal'][closest_score_ind]
        else:
            current_state['keypoints_start'] = prev_state['keypoints_start']
            current_state['start'] = prev_state['start']
    keypoint_plan.append(current_state)
    
    close_to_goal, dist = at_goal(current_state, goal_state)
    _, dist_grasp_goal = at_goal(current_state, grasp_goal)
#     print("dist: " + str(dist))
    print("dist to grasp goal: " + str(dist_grasp_goal))
    
    if close_to_goal or k > max_k or executed_grasp:
        done = True
    else:
        k += 1
    

log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs after pulling: 
dist to grasp goal: 0.04708083659099241
log probs 

In [273]:
plan_fig = plot_keypoint_plan(keypoint_plan)

plan_fig.show()

In [274]:
pull_test = sample_pull_transformation(start_data=keypoint_plan[-2], pull_vae=pull_trans_vae, num_samples=10)
pull_test = transform_keypoints(pull_test, number=10)
plot_keypoint_goal(pull_test).show()

In [291]:
grasp_test = sample_grasp_transformation(start_data=keypoint_plan[-2], grasp_vae=grasp_trans_vae, grasp_clf=grasp_trans_clf, num_samples=10)
grasp_test = transform_keypoints(grasp_test, number=10)
plot_keypoint_goal(grasp_test).show()

# vis_palms(grasp_test, name='grasp', goal='transformation', goal_number=10).show(viewer='gl')

In [288]:
grasp_contact_test = empty_state()
ind = 3
grasp_contact_test['start'] = grasp_test['start']
grasp_contact_test['transformation_corrected'] = [grasp_test['transformation_corrected'][ind]]
grasp_contact_test['keypoints_start'] = grasp_test['keypoints_start']
grasp_contact_test['keypoints_goal_corrected'] = grasp_test['keypoints_goal_corrected'][ind]

grasp_contact_test_out = sample_contact(grasp_palm_vae, grasp_contact_test, primitive='grasp', num_samples=10)

vis_palms(grasp_contact_test_out, name='grasp', goal='transformation', palm_number=10).show(viewer='gl')

realsense_box_experiments.stl
table_top.stl
mpalms_all_coarse.stl
mpalms_all_coarse.stl:EZJCRMSKUZLP
mpalms_all_coarse.stl:EZIOMBXAN3PP
mpalms_all_coarse.stl:EZIGHHAIO9BN
mpalms_all_coarse.stl:EZJZHMH1ZLPP
mpalms_all_coarse.stl:EZJGH4GUEZPV
mpalms_all_coarse.stl:EZL2OCE6KWQP
mpalms_all_coarse.stl:EZL5ZZXFXXFU
mpalms_all_coarse.stl:EZIALAB0TLAP
mpalms_all_coarse.stl:EZLMYXMQ7YTT
mpalms_all_coarse.stl:EZIAFCJMBQLX
mpalms_all_coarse.stl:EZLANZCVTAE0
mpalms_all_coarse.stl:EZINCVQFPRVP
mpalms_all_coarse.stl:EZKSHFAQIJZF
mpalms_all_coarse.stl:EZLGQZSPECPP
mpalms_all_coarse.stl:EZIWIOC3LAKP
mpalms_all_coarse.stl:EZKGTQMRRHUP
mpalms_all_coarse.stl:EZIOJMKXVWXT
mpalms_all_coarse.stl:EZL35F0EZAPN
mpalms_all_coarse.stl:EZLZ4NBZZQTT
realsense_box_experiments.stl:EZJSUW7JIDXV


SceneViewer(width=1800, height=1350)

In [195]:
grasp_fig = plot_keypoint_goal(grasp_data_eval)
grasp_fig.show()

# Analyze latent space

In [292]:
x_embedded = TSNE(n_components=2).fit_transform(x_train)

In [293]:
x_sampled = grasp_trans_clf.sample(5000)[0]

print(x_sampled)

print(x_sampled.shape)
print(x_train.shape)

[[-0.00048812 -0.00275932 -0.00041756]
 [-0.00077224 -0.00626889  0.00092097]
 [-0.00078491 -0.00101774  0.00017297]
 ...
 [ 0.00241952  0.01025966 -0.00205008]
 [ 0.00309835  0.01086254 -0.00061242]
 [ 0.00187404  0.00739138  0.00015471]]
(5000, 3)
(4901, 3)


In [ ]:
x_sampled_embedded = TSNE(n_components=2).fit_transform(x_sampled)

In [ ]:
plt.figure()
plt.scatter(x_embedded[:, 0], x_embedded[:, 1], color='red')
plt.scatter(x_sampled_embedded[:, 0], x_sampled_embedded[:, 1], color='blue')
plt.show()

# Overfit to single batch

In [ ]:
grasp_trans_vae = GoalVAE(31, 7, 3, 24, [64, 32], 0.0003)
print(grasp_trans_vae.encoder)
print(grasp_trans_vae.decoder)
load_net_state(grasp_trans_vae, '/root/training/saved_models/grasp_transformation_cond_start_small_batch_small_model_2/grasp_transformation_cond_start_small_batch_small_model_2_epoch_140.pt')
grasp_trans_vae.encoder.cuda()
grasp_trans_vae.decoder.cuda()

In [ ]:
with open ('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/train/4114/pkl/4114.pkl', 'rb') as f:
    grasp_data_overfit = pickle.load(f)

grasp_data_overfit_copy = copy.deepcopy(grasp_data_overfit)    

In [ ]:
print(grasp_data_overfit['transformation_corrected'])

plot_keypoint_goal(transform_keypoints(grasp_data_overfit)).show()

scene = vis_palms(grasp_data_overfit, name='grasp', goal='transformation')
scene.show(viewer='gl')

In [ ]:
inp0 = torch.from_numpy(grasp_data_overfit['keypoints_start'].flatten().astype(np.float32)).cuda()
inp1 = torch.from_numpy(np.asarray(grasp_data_overfit['transformation_corrected'], dtype=np.float32)).cuda()
inp = torch.cat((inp0, inp1))
z_sample = grasp_trans_vae.encode(inp)[1]

x = torch.from_numpy(grasp_data_overfit['keypoints_start'].flatten().astype(np.float32)).cuda()

trans = grasp_trans_vae.decode(z_sample, x)
print(pose_to_list(trans))

grasp_data_overfit_copy['transformation_corrected'] = pose_to_list(trans)
plot_keypoint_goal(transform_keypoints(grasp_data_overfit_copy)).show()

scene = vis_palms(grasp_data_overfit_copy, name='grasp', goal='transformation')
scene.show(viewer='gl')

In [ ]:
print(grasp_trans_vae.encoder)
print(grasp_trans_vae.decoder)

In [ ]:
print(grasp_trans_vae.encoder.hidden_layers)
print(grasp_trans_vae.decoder.hidden_layers)

In [ ]:
a = [64, 32]
b = copy.deepcopy(a)
b.reverse()

In [ ]:
grasp_data_overfit.keys()

In [ ]:
print(grasp_trans_vae.encode(inp)[1])
print(grasp_trans_vae.encoder.forward(inp))

print(grasp_trans_vae.decode(z_sample, x))
print(grasp_trans_vae.decoder.forward(torch.cat((z_sample, x))))

print(grasp_data_overfit['transformation_corrected'])

trans_inp = torch.from_numpy(np.asarray(grasp_data_overfit['transformation_corrected'], dtype=np.float32)).cuda()
print(grasp_trans_vae.mse(trans[:3], trans_inp[:3]))
# print(trans_inp[3:].expand((2, 4)))
print(grasp_trans_vae.rotation_loss(trans[3:].expand(2, 4), trans_inp[3:].expand(2, 4)))

In [ ]:
grasp_trans_vae.decoder.state_dict().keys()